In [28]:
import pandas as pd 
import os 

# Set directory
os.chdir("/Users/gsgeiger/suspicion_machines_workshop")

# Stats from Dutch national statistics bureau
neighborhood_stats = pd.read_excel("data/01_raw/kerncijfers_wijken_en_buurten_2021.xlsx")

# Synthetic data
synth_df = pd.read_csv("data/01_raw/synth_data.csv")

# Our feature handbook 
feature_handbook = pd.read_excel("data/05_feature_information/feature_handbook.xlsx")


In [29]:

"""
Parses the district from the feature name 
@input raw_str: The raw feature name containing the district
@return: formatted_str: The district name in title case 
"""
def parse_neighborhood(raw_str:str) -> str : 

    if "wijk" in raw_str : 
        district_str = raw_str.split("wijk")[1].strip()

        if district_str == "other" : 
            district_str = "wijk other"
    
    formatted_str = district_str.title()

    # For a few districts, the name doesn't quite match the CBS data so we have to manually adjust 
    if formatted_str == "Kralingen C" : 
        formatted_str = "Kralingen-Crooswijk"
    
    elif formatted_str == "Prins Alexa" : 
        formatted_str = "Prins Alexander"

    elif formatted_str == "Stadscentru" : 
        formatted_str = "Rotterdam Centrum"
    
    elif formatted_str == "Groot Ijsselmonde" : 
        formatted_str = "Groot IJsselmonde"

    elif formatted_str == "Ijsselmonde" : 
        formatted_str = "IJsselmonde"

    return formatted_str

# Filter by category so we only have address features 
address_features = feature_handbook[feature_handbook.Category == "Address"]

# Filter for only address features ie. those containing "wijk" or "buurt"
district_features = address_features[address_features["feature_dutch"].str.contains("wijk")]

# Drop false positive features by filtering by 'recentste' and 'buurt' 
district_features = district_features[district_features["feature_dutch"].str.contains("recentste")]
district_features = district_features[~district_features["feature_dutch"].str.contains("buurt")]

# Create new column district_name with the parsed district name 
district_features["district_name"] = district_features["feature_dutch"].apply(parse_neighborhood)

### Prepare our CBS Data 

We need to make a row for Wijk other in our CBS data. In this case, each column for wijk_other has the median value of all districts that are NOT features in the model. <br>
<br>
The exception is the number of welfare recepients column, where we instead sum the number of welfare recepients in all districts that are NOT features in the model. 

In [30]:
import numpy as np

# Filter out all neighborhoods with 0 people living there 
neighborhood_stats = neighborhood_stats[neighborhood_stats["Bevolking/Aantal inwoners (aantal)"] > 0]

# Create a copy of the neighborhood stats dataframe where we've filtered out all districts that are features in the model
neighborhood_stats_filtered = neighborhood_stats[~neighborhood_stats["Wijken en buurten"].isin(district_features["district_name"])]

# Filter for wijken
wijk_stats = neighborhood_stats_filtered[neighborhood_stats_filtered["Regioaanduiding/Soort regio (omschrijving)"].str.strip() == "Wijk"]

# Create a list of all numeric columns
numeric_columns = neighborhood_stats_filtered.select_dtypes(include=[np.number]).columns

# Drop number of welfare recepients because we want to sum this column 
numeric_columns = numeric_columns.drop(["Sociale zekerheid/Personen per soort uitkering; Bijstand (aantal)"])

# Wijk Median 
wijk_mean = wijk_stats[numeric_columns].median()
wijk_mean_row = pd.DataFrame(wijk_mean).T
wijk_mean_row["Wijken en buurten"] = "Wijk Other"
wijk_mean_row["Sociale zekerheid/Personen per soort uitkering; Bijstand (aantal)"] = wijk_stats["Sociale zekerheid/Personen per soort uitkering; Bijstand (aantal)"].sum()
wijk_mean_row["Regioaanduiding/Gemeentenaam (naam)"] = "Rotterdam"
wijk_mean_row["Regioaanduiding/Soort regio (omschrijving)"] = "Wijk"
wijk_mean_row["Regioaanduiding/Codering (code)"] = None

# Add our wijk_other row to the whole, unfiltered CBS Dataset 
wijk_mean_row = wijk_mean_row[neighborhood_stats_filtered.columns]
neighborhood_stats = pd.concat([neighborhood_stats, wijk_mean_row], ignore_index=True)

# Remove Delfshaven & Feijenoord copies because they are both wjik and buurt. In this case, we only want the wijk 
neighborhood_stats = neighborhood_stats[~((neighborhood_stats['Wijken en buurten'].str.strip() == 'Delfshaven') 
                                          & (neighborhood_stats['Regioaanduiding/Soort regio (omschrijving)'].str.strip() == 'Buurt'))]

neighborhood_stats = neighborhood_stats[~((neighborhood_stats['Wijken en buurten'].str.strip() == 'Feijenoord') 
                                          & (neighborhood_stats['Regioaanduiding/Soort regio (omschrijving)'].str.strip() == 'Buurt'))]


### Merge our neighborhood statistics with our neighborhood features dataframe 

In [31]:
algo_district_stats = pd.merge(district_features,neighborhood_stats,left_on="district_name",right_on="Wijken en buurten",how="left")

print(algo_district_stats.head())


  Unnamed: 0                    feature_dutch_   
0        NaN   adres recentste wijk feijenoord  \
1        NaN   adres recentste wijk delfshaven   
2        NaN  adres recentste wijk prins alexa   
3        NaN     adres recentste wijk charlois   
4        NaN        adres recentste wijk noord   

                      feature_dutch   
0   adres recentste wijk feijenoord  \
1   adres recentste wijk delfshaven   
2  adres recentste wijk prins alexa   
3     adres recentste wijk charlois   
4        adres recentste wijk noord   

                              feature_english           variable_generation   
0  address most recent district of Feijenoord  [DUMMY] adres_recentste_wijk  \
1     address most recent Delfshaven district  [DUMMY] adres_recentste_wijk   
2    address most recent district Prins Alexa  [DUMMY] adres_recentste_wijk   
3       address most recent Charlois district  [DUMMY] adres_recentste_wijk   
4          address most recent district Noord  [DUMMY] adres_recentst

### Calculate our Representation Statistics 

In [32]:

"""
@input: The Dutch district feature name 
@input: Synthetic training data 
@return: The total number of welfare recepients with that district 
"""
def district_count_training_data(district:str,synth_data:pd.DataFrame) -> int : 
    district_underscore = district.replace(" ","_")

    return synth_data[district_underscore].sum()


# Calculate the count of each district in the training data 
algo_district_stats["district_training_data_count"] = algo_district_stats["feature_dutch"].apply(district_count_training_data,args=(synth_df,))

"""
There are a lot of rows in the synthetic data that don't seem to have a 1 in any wijk, so we should remove them from the N to test 
proportionality correctly 
"""
training_N_no_na = algo_district_stats["district_training_data_count"].sum()

# The number of welfare recepients in Rotterdam is on the second row of the dataframe
total_welfare_recepients = neighborhood_stats["Sociale zekerheid/Personen per soort uitkering; Bijstand (aantal)"][1]

# Calculate the share of welfare recepients in a neighborhood in the real world 
algo_district_stats["welfare_share_real"] = algo_district_stats["Sociale zekerheid/Personen per soort uitkering; Bijstand (aantal)"].apply(
    lambda district_welfare_n : district_welfare_n / total_welfare_recepients
)

# Calculate the share of welfare recepients in a neighborhood in the training data 
algo_district_stats["welfare_share_training_data"] = algo_district_stats["district_training_data_count"].apply(
    lambda district_training_n : district_training_n / training_N_no_na
)

# Calculate over/under representation 
algo_district_stats["representation"] = algo_district_stats["welfare_share_training_data"] / algo_district_stats["welfare_share_real"]

### Save our dataframe 

In [33]:
algo_district_stats.drop(
    columns=["Unnamed: 0",
             "feature_dutch_",
             "Notes",
             "Understand? ",
             "Wijken en buurten",
             "Config",
             "feature_simplified "],
    inplace=True)

# Export all district stats 
neighborhood_stats.to_excel("data/02_intermediate/all_district_stats.xlsx")

# Export algorithm district stats 
algo_district_stats.to_excel("data/02_intermediate/algo_district_stats.xlsx",index=False)

